# TP 2

## Imports

### imports base

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import matplotlib
import plotly as py
import plotly.express as px 
import matplotlib.ticker as ticker
plt.style.use('default')
import plotly.offline as opy
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
from datetime import datetime
from datetime import timedelta

import math

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
!pip3 install category_encoders

     |████████████████████████████████| 81kB 4.7MB/s 


In [ ]:
import category_encoders as ce

In [ ]:
!pip3 install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=89767499baaea403012fc2ea70ce564faaceef5dfb4e34036bf520c80a16ae82
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
from bayes_opt import BayesianOptimization

### catboost

In [ ]:
#Ejecutar si no funciona el import
!pip3 install catboost

     |████████████████████████████████| 65.8MB 69kB/s 


In [ ]:
from catboost import CatBoostClassifier, Pool, cv

### xgboost

In [ ]:
!pip3 install xgboost

In [ ]:
import xgboost as xgb

### lightgbm

In [ ]:
!pip3 install lightgbm

In [ ]:
import lightgbm as lgb
#from lightgbm import LGBMClassifier

### ramdom forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

## lectura de datos

In [ ]:
entrenamiento = pd.read_csv('Training.csv')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [ ]:
testeo = pd.read_csv('Test.csv')
testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

In [ ]:
#hemisferios = pd.read_csv('hemisferios.csv')
#hemisferios = convertirATipo(hemisferios,['Billing_Country','hemisferio'],'category')
#hemisferios.info()

In [ ]:
#prueva de supuestos
supuestos = pd.read_csv('EvaluacionTemporal.csv',index_col='Opportunity_ID')
supuestos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Target  1567 non-null   float64
dtypes: float64(1)
memory usage: 24.5 KB


## prosesamiento de los df

### modificaciones de oportunidad

In [ ]:
def convertirATiempo(df,lista):
    for columna in lista:
        df[columna] = pd.to_datetime(df[columna])
    return df

def convertirATipo(df,lista,tipo):
    for columna in lista:
        df[columna] = df[columna].astype(tipo)
    return df

def convertirAFlotante(df,lista):
    for columna in lista:
        df[columna] = df[columna].apply(lambda x: None if (x == "None" or x == "Other") else float(x))
    return df

def establecerTipos(df):
    """cambia los tipos de las columnas"""
    
    #fechas
    listaDeColumnasFechas = ['Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Last_Modified_Date','Delivery_Year','Planned_Delivery_End_Date','Planned_Delivery_Start_Date']
    df = convertirATiempo(df,listaDeColumnasFechas)
    #enteros
    listaDeColumnasEnteras = ['ID','Opportunity_ID']
    convertirATipo(df,listaDeColumnasEnteras,'int')
    #decimales
    listaDeColumnasEnteras = ['Price','ASP','ASP_(converted)','Total_Taxable_Amount','Total_Amount']
    convertirAFlotante(df,listaDeColumnasEnteras)
    #booleanos
    listaDeColumnasBooleanas = ['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved','Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']
    convertirATipo(df,listaDeColumnasBooleanas,'bool')
    #categoricas
    listaDeColumnasCategoricas = ['Product_Category_B','Product_Family','Region','Territory','Bureaucratic_Code','Source ','Billing_Country','Account_Name','Opportunity_Name']
    convertirATipo(df,listaDeColumnasCategoricas,'category')
    return df

In [ ]:
def agregarHemisferios(df):
  #df.reset_index()
  df = df.merge(hemisferios,how='left',on='Billing_Country')
  df['hemisferio'] = df['hemisferio'].apply(lambda x: 'Desconocido' if (x != 'Norte' and x != 'Sur') else x)
  convertirATipo(df,['hemisferio','Billing_Country'],'category')
  #df.set_index('Opportunity_ID')
  return df

In [ ]:
def eliminarColumnasSinImportancia(df):
    #"Last_Activity" y "Actual_Delivery_Date" son puros nulos
    #"Submitted_for_Approval" puros valores nulos
    #"Prod_Category_A" tiene todos sus valores iguales
    df = df.drop(['Submitted_for_Approval','Last_Activity','Actual_Delivery_Date','Prod_Category_A'], axis=1)
    return df

def agregarColumnas(df):
    df['Closed'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': True, 'Negotiation':False,'Proposal':False,'Qualification':False})#REVISAR
    df['Won'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': False, 'Negotiation':False,'Proposal':False,'Qualification':False})
    convertirATipo(df,['Stage'],'category')
    return df

def correccionDeTerritorios(df):
    #se refiere a diferentes partes de usa
    df = df.replace({'Territory':{'Central US':'USA','NE America':'USA', 'NW America':'USA','SW America':'USA','SE America':'USA'}})
    #
    df = df.replace({'Territory':{'KSA':'Saudi Arabia'}})
    #No se ha encontrado la dorma de los siguientes datos 'Kosovo','South America', 'South East Asia', 'UAE (Dubai)', 'UAE (Others)' (en el caso de kosovo no lo grafica)
    return df

def preparacionDelDataFrame(df,entrenamiento=True):
    df = eliminarColumnasSinImportancia(df)
    df = establecerTipos(df)
    df = correccionDeTerritorios(df)
    df.rename(columns = {'Source ':'Source'}, inplace = True)
    #df = agregarHemisferios(df)#desactivado desde el readcsv
    if (entrenamiento):
      df = agregarColumnas(df)
    return df

### por oportunidad

In [ ]:
def desacerseDeNulosDateTime(df,columna):
  df[columna] = df[columna].apply(lambda x: datetime.max if (x is None or x is np.NaN or x is pd.NaT) else x)
  return df

def descomposicionDeFechas(df,lista):
  """se nesesitan enteros no fechas"""
  for columna in lista:
    anio = columna + '_anio'
    df[anio] = df[columna].apply(lambda x: x if x is None else x.year)
    mes = columna + '_mes'
    df[mes] = df[columna].apply(lambda x: x if x is None else x.month)
    dia = columna + '_dia'
    df[dia] = df[columna].apply(lambda x: x if (x is None) else x.day)
  df = df.drop(lista, axis=1)
  return df

def establecerEstacion(df,columnaFecha):
  estacion = columnaFecha + '_Estacion'
  df[estacion] = 'Desconocido'
  df['temporalEstaciones'] = df[columnaFecha].apply(lambda x: 'Desconocido' if x is None else estaciones(x,True))
  df.loc[df['hemisferio'] == 'Norte' , estacion] = df['temporalEstaciones']
  df['temporalEstaciones'] = df[columnaFecha].apply(lambda x: 'Desconocido' if x is None else estaciones(x,False))
  df.loc[df['hemisferio'] == 'Sur' , estacion] = df['temporalEstaciones']
  df = df.drop(['temporalEstaciones'], axis=1)
  convertirATipo(df,[estacion],'category')
  return df

def estaciones(fecha,norte):
  if (norte):
    estaciones = ['primavera','verano','otoño','invierno']
  else:
    estaciones = ['otoño','invierno','primavera','verano']
  if (fecha.month == 12 and fecha.day >= 21 or fecha.month <= 3 and fecha.day < 21):
    return estaciones[3]
  elif (fecha.month == 9 and fecha.day >= 21 or fecha.month > 9):
    return estaciones[2]
  elif (fecha.month == 6 and fecha.day >= 21 or fecha.month > 6):
    return estaciones[1]
  elif (fecha.month == 3 and fecha.day >= 21 or fecha.month > 3):
    return estaciones[0]
  else:
    return 'Desconocido'

def diferenciaDeFechasEnDias(diferenciaDeFechas):
  return (diferenciaDeFechas.year * 365 + diferenciaDeFechas.month * 30 + diferenciaDeFechas.day)

def diferenciaEntreFechas(df,colFechaInicial,colFechaFinal):
  df['TiempoDeDiferencia'] = (df[colFechaFinal] - df[colFechaInicial])
  df['TiempoDeDiferencia'] = pd.to_numeric(df['TiempoDeDiferencia'].dt.days, downcast='integer')#revisar si se conviete totalmente a dias
  #df['TiempoDeDiferencia'] = df['TiempoDeDiferencia'].apply(lambda x: diferenciaDeFechasEnDias(x))
  df = df.drop([colFechaInicial,colFechaFinal], axis=1)
  return df

In [ ]:
def dataEntrenamiento(df):
  data = df.groupby('Opportunity_ID').agg({'Won':'sum','Closed':'sum',
                                           'modificaciones':'sum',
                                           #'Won':'sum','modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           #'Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first',
                                           'Billing_Country':'first',
                                           'Planned_Delivery_End_Date':'max',
                                           'Planned_Delivery_Start_Date':'min',
                                           'Delivery_Terms':'first',
                                           'Product_Family':'first',
                                           'TRF':'mean'
                                           #'hemisferio':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data

def dataTesteo(df):
  data = df.groupby('Opportunity_ID').agg({'modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           #'Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first',
                                           'Billing_Country':'first',
                                           'Planned_Delivery_End_Date':'max',
                                           'Planned_Delivery_Start_Date':'min',
                                           'Delivery_Terms':'first',
                                           'Product_Family':'first',
                                           'TRF':'mean'
                                           #'hemisferio':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data  

def agrupacionDeOportunidades(df,entrenamiento=True):
  df['modificaciones'] = 1
  if (entrenamiento):
    data = dataEntrenamiento(df)
    #Solo me interesa saber si se cerro con exito
    #data['Closed'] = data['Closed'].agg(lambda x: 1 if x > 0 else 0)
    data['Won'] = data['Won'].agg(lambda x: 1 if x > 0 else 0)
    #data['Lost'] = data['Closed'] - data['Won']#si es won es closed pero si es closed y no es won es perdido
    #data['Lost'] = data['Lost'].agg(lambda x: 1 if x > 0 else 0)

    #REVISAR la importancia del caso no cerrado
    data = data[data['Closed'] > 0]#no interesan los que no se cerraron, dado que no se cerraron no aportan
    data = data.drop(['Closed'], axis=1)
  else:
    data = dataTesteo(df)

  #data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: None if x == 0 else x)
  #data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: None if x == 0 else x)
  #data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: None if x == 0 else x)
  #data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: None if x == 0 else x)
  """
  data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: 1 if x > 0 else 0)
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: 1 if x > 0 else 0)

  #Condensando columnas
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Quote_Appr'] + data['Pricing, Delivery_Terms_Approved']
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approval'] + data['Bureaucratic_Code_0_Approved']
  data = data.drop(['Bureaucratic_Code_0_Approval','Pricing, Delivery_Terms_Quote_Appr'], axis=1)

  data['Pricing, Delivery_Terms_Condensado'] = data['Pricing, Delivery_Terms_Quote_Appr'] + data['Pricing, Delivery_Terms_Approved']
  data['Pricing, Delivery_Terms_Condensado'] = data['Pricing, Delivery_Terms_Condensado'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved'], axis=1)

  data['Bureaucratic_Code_0_Condensado'] = data['Bureaucratic_Code_0_Approval'] + data['Bureaucratic_Code_0_Approved']
  data['Bureaucratic_Code_0_Condensado'] = data['Bureaucratic_Code_0_Condensado'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved'], axis=1)
  #data = data.replace({'Won':})

  data['aprobacion'] = data['Pricing, Delivery_Terms_Condensado'] + data['Bureaucratic_Code_0_Condensado']
  data['aprobacion'] = data['aprobacion'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Pricing, Delivery_Terms_Condensado','Bureaucratic_Code_0_Condensado'], axis=1)

  """

  #categoricas
  #listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Delivery_Quarter']
  listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country',
                                'Delivery_Quarter','Product_Type','Product_Category_B',
                                'Delivery_Terms','Product_Family']
  convertirATipo(data,listaDeColumnasCategoricas,'category')

  #data = establecerEstacion(data,'Opportunity_Created_Date')
  #data = establecerEstacion(data,'Quote_Expiry_Date')
  #data = desacerseDeNulosDateTime(data,'Quote_Expiry_Date')#si no hay fecha de expiracion se pre supone que es infinita

  listaDeValoresDeFechas = ['Last_Modified_Date','Opportunity_Created_Date',
                            'Account_Created_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
  data = descomposicionDeFechas(data,listaDeValoresDeFechas)

  #enteros
  #listaDeColumnasEnteras = ['Quote_Expiry_Date_anio','Quote_Expiry_Date_mes','Quote_Expiry_Date_dia']
  #convertirATipo(data,listaDeColumnasEnteras,'int')
  #data = agregarHemisferios(data)

  #data = data.drop(['hemisferio'], axis=1)

  #data = diferenciaEntreFechas(data,'Planned_Delivery_Start_Date','Planned_Delivery_End_Date')

  return data

las siguientes columnas no fueron probadas hay que analizarlas

Posibles columnas a usar:

- Bureaucratic_Code
- Delivery_Year (posibles datos repetidos)
- Month (posibles datos repetidos)
- Opportunity_Owner
- Opportunity_Type
- Product_Family
- Product_Name
- Quote_Type (muy desbalanceado)

Revisar el contexto

- Total_Amount_Currency
- Total_Taxable_Amount
- Total_Taxable_Amount_Currency
- ASP
- ASP_(converted)_Currency
- ASP_Currency


las columnas con gran catidad de nones parecen afectar positivamente el resultado hay que revisarlas una por una

- Currency: Segun prueba de supuestos empeora el resultado
- Brand: Segun prueba de supuestos empeora el resultado ligeramente
- Size: Segun prueba de supuestos empeora el resultado ligeramente
- Source: Segun prueba de supuestos empeora el resultado

### categoricas a enteros (caso de que el sistema predictor no reconosca category)

In [ ]:
def convertirCategoryEnEnteros(df1,df2,columna):
  encoder = ce.BinaryEncoder(cols=[columna])
  encoder.fit(df1)
  #encoder.fit(df2)
  df1 = encoder.transform(df1)
  df2 = encoder.transform(df2)
  return df1,df2


## Ejecucion de cambios

### ejecucion por modificaciones de oportunidad

In [ ]:
entrenamientoProcesado = preparacionDelDataFrame(entrenamiento)
entrenamientoProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16947 non-null  int64         
 1   Region                              16947 non-null  category      
 2   Territory                           16947 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  bool          
 4   Pricing, Delivery_Terms_Approved    16947 non-null  bool          
 5   Bureaucratic_Code_0_Approval        16947 non-null  bool          
 6   Bureaucratic_Code_0_Approved        16947 non-null  bool          
 7   Bureaucratic_Code                   16947 non-null  category      
 8   Account_Created_Date                16947 non-null  datetime64[ns]
 9   Source                              16947 non-null  category      
 10  Billing_Country       

In [ ]:
testeoProsesado = preparacionDelDataFrame(testeo,False)
testeoProsesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   category      
 2   Territory                           2551 non-null   object        
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   bool          
 4   Pricing, Delivery_Terms_Approved    2551 non-null   bool          
 5   Bureaucratic_Code_0_Approval        2551 non-null   bool          
 6   Bureaucratic_Code_0_Approved        2551 non-null   bool          
 7   Bureaucratic_Code                   2551 non-null   category      
 8   Account_Created_Date                2551 non-null   datetime64[ns]
 9   Source                              2551 non-null   category      
 10  Billing_Country         

### ejecucion por oportunidad

In [77]:
agrupacionEntrenamiento = agrupacionDeOportunidades(entrenamientoProcesado)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Won                                 9791 non-null   int64   
 1   modificaciones                      9791 non-null   int64   
 2   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64   
 3   Pricing, Delivery_Terms_Approved    9791 non-null   int64   
 4   Bureaucratic_Code_0_Approval        9791 non-null   int64   
 5   Bureaucratic_Code_0_Approved        9791 non-null   int64   
 6   Total_Amount                        9791 non-null   float64 
 7   Region                              9791 non-null   category
 8   Territory                           9791 non-null   category
 9   ASP_(converted)                     9791 non-null   float64 
 10  Account_Type                        9791 non-null   category
 11  Delivery_Quarter             

In [78]:
agrupacionTesteo = agrupacionDeOportunidades(testeoProsesado,False)
agrupacionTesteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   modificaciones                      1567 non-null   int64   
 1   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   int64   
 2   Pricing, Delivery_Terms_Approved    1567 non-null   int64   
 3   Bureaucratic_Code_0_Approval        1567 non-null   int64   
 4   Bureaucratic_Code_0_Approved        1567 non-null   int64   
 5   Total_Amount                        1567 non-null   float64 
 6   Region                              1567 non-null   category
 7   Territory                           1567 non-null   category
 8   ASP_(converted)                     1567 non-null   float64 
 9   Account_Type                        1567 non-null   category
 10  Delivery_Quarter                    1567 non-null   category
 11  Product_Type             

### separacion de resultados de entrenamiento

In [79]:
entrenamientoResultado = agrupacionEntrenamiento[['Won']]
entrenamientoResultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Won     9791 non-null   int64
dtypes: int64(1)
memory usage: 153.0 KB


In [80]:
agrupacionEntrenamiento = agrupacionEntrenamiento.drop(['Won'], axis=1)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   modificaciones                      9791 non-null   int64   
 1   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64   
 2   Pricing, Delivery_Terms_Approved    9791 non-null   int64   
 3   Bureaucratic_Code_0_Approval        9791 non-null   int64   
 4   Bureaucratic_Code_0_Approved        9791 non-null   int64   
 5   Total_Amount                        9791 non-null   float64 
 6   Region                              9791 non-null   category
 7   Territory                           9791 non-null   category
 8   ASP_(converted)                     9791 non-null   float64 
 9   Account_Type                        9791 non-null   category
 10  Delivery_Quarter                    9791 non-null   category
 11  Product_Type                 

## Prueva

### prueva todo igual

In [ ]:
prueva = agrupacionTesteo.reset_index()
prueva = prueva[['Opportunity_ID']]
prueva['prediccion'] = 0.5
prueva
prueva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Opportunity_ID  1567 non-null   int64
 1   prediccion      1567 non-null   int64
dtypes: int64(2)
memory usage: 24.6 KB


In [ ]:
prueva = agrupacionTesteo
prueva['Target'] = 0.5
prueva = prueva[['Target']]
prueva.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Target  1567 non-null   float64
dtypes: float64(1)
memory usage: 24.5 KB


In [ ]:
from google.colab import files

prueva.to_csv('prueva.csv')
#files.download('resultado.csv')

### prueva supuestamete bien (demasiado arbitraria para funcionar)

In [192]:
serieDiferenciasDeSupuestos = dataResultante['Target'] - supuestos['Target']
serieDiferenciasDeSupuestos = serieDiferenciasDeSupuestos.agg(lambda x: abs(x))
#valores a vencer
#0.2570274780297901 0.06006494971598539 0.24508151647153115
#0.2507114153988768 0.06688690577185248 0.25862502928342507
#0.25239717054295296 0.06270391586504633 0.25040749961821496
print(serieDiferenciasDeSupuestos.mean(),serieDiferenciasDeSupuestos.var(),serieDiferenciasDeSupuestos.std())

0.2834110466187937 0.0455177037706077 0.21334878431949805


In [ ]:

print(serieDiferenciasDeSupuestos.mean(),serieDiferenciasDeSupuestos.var(),serieDiferenciasDeSupuestos.std())

0.2507114153988768 0.06688690577185248 0.25862502928342507


In [ ]:
supuestos['TargetBinario'] = supuestos['Target'].agg(lambda x: 1 if x > 0.5 else 0)

In [193]:
serieDiferenciasDeSupuestos = dataResultante['Target'] - supuestos['TargetBinario']
serieDiferenciasDeSupuestos = serieDiferenciasDeSupuestos.agg(lambda x: abs(x))
#0.29328282239325076 0.066288348418729 0.25746523730152193
print(serieDiferenciasDeSupuestos.mean(),serieDiferenciasDeSupuestos.var(),serieDiferenciasDeSupuestos.std())

0.327814826632631 0.04688975465941205 0.21654042269149668


In [194]:
dataResultante['Target'].agg(lambda x: 1 if x < 0.55 and x > 0.45 else 0).value_counts()

0    1419
1     148
Name: Target, dtype: int64

In [ ]:
#prueva de supuestos
#vencer = pd.read_csv('valoresAVencer.csv',index_col='Opportunity_ID')
#vencer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Target  1567 non-null   float64
dtypes: float64(1)
memory usage: 24.5 KB


## Resultado

In [ ]:
def generarResultado(df,prediccion):
  resultado = df
  resultado['Target'] = prediccion
  resultado = resultado[['Target']]
  resultado.to_csv('resultado.csv')
  return resultado

In [ ]:
def auxiliarUnificarValor(arreglo):
  return arreglo[:,0]

## Catboost

### grid search

In [ ]:
def GridSearchCatboost(train_data_o,train_labels,cat_features):
  """clf = CatBoostClassifier()
  params = {'iterations': [500],
            'depth': [4, 5, 6],
            #'loss_function': ['Logloss', 'CrossEntropy'],
            'l2_leaf_reg': np.logspace(-10, 1, 50),
            'leaf_estimation_iterations': [10],
            'learning_rate': [0.5,0.25,0.05],
            'rsm' : [0.7,0.5]
            #'eval_metric': ['Accuracy'],
            #'use_best_model': ['True'],
            #'logging_level':['Silent'],
            #'random_seed': [42]
          }
  scorer = make_scorer(accuracy_score)
  clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)
  clf_grid.fit(X_train_enc, y_train,cat_features)
  best_param = clf_grid.best_params_
  return best_param"""
  model = CatBoostClassifier(loss_function='Logloss')
  #model = CatBoostClassifier(loss_function='CrossEntropy')

  train_data = train_data_o
  # convert categorical columns to integers
  for header in cat_features:
    train_data[header] = train_data[header].astype('category').cat.codes

  grid = {'learning_rate': list(np.arange(0.05,0.51,0.05)),
          'depth': list(range(6,13,2)),
          'l2_leaf_reg': list(np.arange(2,5.1,0.5)),
          'iterations': list(range(10,101,15)),
          'rsm':list(np.arange(0.1,1.1,0.1))
          }

  grid_search_result = model.grid_search(grid, 
                                        X=train_data, 
                                        y=train_labels,
                                        cv=10,
                                        plot=True)
  
  return grid_search_result

In [ ]:
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                      'Account_Type','Delivery_Quarter','Billing_Country']
#listaDeCategoricas = []
valores = GridSearchCatboost(agrupacionEntrenamiento,entrenamientoResultado,listaDeCategoricas)
valores

### bayesian

In [ ]:
def catboost_classifier(learning_rate, depth, l2_leaf_reg,iterations,rsm):
    params = {
        "loss_function": "Logloss",
        #"eval_metric" : "AUC", 
        "depth" : int(depth),
        "l2_leaf_reg" : l2_leaf_reg,
        "learning_rate" : learning_rate,
        #"random_state" : 42,
        "rsm" : rsm,
        "iterations": int(iterations)
        #"logging_level" : "Silent",
        #"thread_count": 24,
        #"num_boost_round": int(num_boost_round)
    }
    X_train = agrupacionEntrenamiento
    y_train = entrenamientoResultado
    categorical_features = ['Region','Territory','Product_Type','Product_Category_B',
                            'Account_Type','Delivery_Quarter','Billing_Country']

    train_data = Pool(data=X_train, label=y_train, 
                               cat_features=categorical_features)
    
    cv_result = cv(train_data,
                       params,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['train-Logloss-mean'].iloc[-1]

def bayesianoCatboost():
  grid = {'learning_rate': (0.01,0.10),
          'depth': (8,14),
          'l2_leaf_reg': (0.1,3.0),
          'iterations': (10,100),
          'rsm':(0.01,1.0)
          }
  catboostBO = BayesianOptimization(catboost_classifier, grid)

  return catboostBO.maximize(n_iter=15, init_points=2)

In [ ]:
bayesianoCatboost()
#catboost_classifier(0.05,10,3.5,20,0.5)

|   iter    |  target   |   depth   | iterat... | l2_lea... | learni... |    rsm    |
-------------------------------------------------------------------------------------
0:	learn: 0.6794059	test: 0.6801177	best: 0.6801177 (0)	total: 154ms	remaining: 12.2s
1:	learn: 0.6666912	test: 0.6682458	best: 0.6682458 (1)	total: 319ms	remaining: 12.5s
2:	learn: 0.6540947	test: 0.6565980	best: 0.6565980 (2)	total: 475ms	remaining: 12.2s
3:	learn: 0.6418077	test: 0.6448695	best: 0.6448695 (3)	total: 635ms	remaining: 12.1s
4:	learn: 0.6289087	test: 0.6325313	best: 0.6325313 (4)	total: 791ms	remaining: 11.9s
5:	learn: 0.6171969	test: 0.6210287	best: 0.6210287 (5)	total: 930ms	remaining: 11.5s
6:	learn: 0.6061907	test: 0.6105018	best: 0.6105018 (6)	total: 1.08s	remaining: 11.3s
7:	learn: 0.5972383	test: 0.6014687	best: 0.6014687 (7)	total: 1.24s	remaining: 11.2s
8:	learn: 0.5875914	test: 0.5920759	best: 0.5920759 (8)	total: 1.4s	remaining: 11.1s
9:	learn: 0.5771851	test: 0.5821646	best: 0.5821646 (9)

### prediccion

In [ ]:
def catboost(train_data_original,train_labels_original,cat_features,eval_data_original):
  # Inicializador de CatBoostClassifier
  model = CatBoostClassifier(iterations=80,
                             learning_rate=0.5,
                             depth=8,
                             l2_leaf_reg=3,
                             rsm=0.5,
                             grow_policy='SymmetricTree'#'SymmetricTree', 'Depthwise' y 'Lossguide'
                             #random_seed=2
                             #use_best_model=True,#Requiere data de evalucion
                             #eval_set=supuestos
                             )
  #parametros importantes por agregar: l2_leaf_reg, rsm
  #para tipos de category: cat_features, one_hot_max_size
  train_data = train_data_original.copy(True)
  eval_data = eval_data_original.copy(True)
  train_labels = train_labels_original.copy(True)

  # Entrenamiento del modelo
  #model.fit(train_data, train_labels, cat_features)

  eval_dataset = Pool(data=eval_data,
                      label=supuestos[['TargetBinario']],
                      cat_features=cat_features)
  model.fit(train_data, train_labels, cat_features, use_best_model=True, eval_set=eval_dataset)

  # obtener prediccion
  preds_class = model.predict(eval_data)

  # obtener probabilidad de prediccion
  #preds_proba = model.predict_proba(eval_data)

  # obtener RawFormulaVal de la prediccion
  #preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')

  #return resultado(eval_data,preds_proba)
  importancia = model.get_feature_importance(type= "FeatureImportance",prettified=True)
  return preds_class,importancia

In [ ]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B']
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter']
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                      'Account_Type','Delivery_Quarter','Billing_Country',
                      'Delivery_Terms','Product_Family']
#listaDeCategoricas = []
res,importancia = catboost(agrupacionEntrenamiento,entrenamientoResultado,listaDeCategoricas,agrupacionTesteo)
res

0:	learn: 0.5209898	test: 0.5233887	best: 0.5233887 (0)	total: 16.9ms	remaining: 1.33s
1:	learn: 0.4724866	test: 0.4994176	best: 0.4994176 (1)	total: 33.5ms	remaining: 1.31s
2:	learn: 0.4367016	test: 0.4936629	best: 0.4936629 (2)	total: 49ms	remaining: 1.26s
3:	learn: 0.4085372	test: 0.4836845	best: 0.4836845 (3)	total: 64.4ms	remaining: 1.22s
4:	learn: 0.3901452	test: 0.4718065	best: 0.4718065 (4)	total: 80.7ms	remaining: 1.21s
5:	learn: 0.3769580	test: 0.4606440	best: 0.4606440 (5)	total: 94.6ms	remaining: 1.17s
6:	learn: 0.3645165	test: 0.4627431	best: 0.4606440 (5)	total: 110ms	remaining: 1.15s
7:	learn: 0.3567527	test: 0.4604855	best: 0.4604855 (7)	total: 127ms	remaining: 1.14s
8:	learn: 0.3446983	test: 0.4780993	best: 0.4604855 (7)	total: 142ms	remaining: 1.12s
9:	learn: 0.3317255	test: 0.4901607	best: 0.4604855 (7)	total: 158ms	remaining: 1.1s
10:	learn: 0.3231400	test: 0.4992002	best: 0.4604855 (7)	total: 174ms	remaining: 1.09s
11:	learn: 0.3166112	test: 0.5069198	best: 0.46048

array([0, 0, 1, ..., 0, 1, 0])

In [ ]:
resultado = auxiliarUnificarValor(res)
#resultado = res
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.41593382, 0.3370681 , 0.63055812, ..., 0.30270279, 0.71528178,
       0.16628556])

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0
10690,0
10691,1
10692,1
10693,1


In [170]:
importancia

,Feature Id,Importances
0,TRF,14.65
1,Product_Type,10.33
2,"Pricing, Delivery_Terms_Approved",9.03
3,Product_Category_B,7.26
4,Region,6.44
5,Planned_Delivery_End_Date_dia,5.60
6,Account_Created_Date_dia,3.96
7,Product_Family,3.82
8,Planned_Delivery_Start_Date_anio,3.81
9,Opportunity_Created_Date_anio,3.41


## XGBoost

In [76]:
def xgboost(train_set, train_label,listaCategorica,test_set,masImportantes=0):
  xgb_reg = xgb.XGBClassifier(max_depth = 8, 
                           colsample_bytree = 0.5, 
                           learning_rate = 0.3, 
                           alpha = 8,
                           min_child_weight = 0.5,
                           n_estimators = 200,
                           objetive = 'reg:squarederror')
  xgb_reg.fit(train_set, train_label)
  predicts = xgb_reg.predict_proba(test_set)
  if (masImportantes > 0):
    xgb.plot_importance(xg_reg, max_num_features=masImportantes)
  return predicts




In [82]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B']
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter']
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
                      'Account_Type','Delivery_Quarter','Billing_Country',
                      'Delivery_Terms','Product_Family']
#listaDeCategoricas = []
agrupacionEntrenamientoXGB = agrupacionEntrenamiento.copy(True)
agrupacionTesteoXGB = agrupacionTesteo.copy(True)

# convert categorical columns to integers 
#usando la funcion de catboost
for header in listaDeCategoricas:
  agrupacionEntrenamientoXGB[header] = agrupacionEntrenamientoXGB[header].astype('category').cat.codes
  agrupacionTesteoXGB[header] = agrupacionTesteoXGB[header].astype('category').cat.codes
#usando metodo personalizado
#for columna in listaDeCategoricas:
#  agrupacionEntrenamientoXGB,agrupacionTesteoXGB = convertirCategoryEnEnteros(agrupacionEntrenamientoXGB,agrupacionTesteoXGB,columna)


agrupacionEntrenamientoXGB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   modificaciones                      9791 non-null   int64  
 1   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64  
 2   Pricing, Delivery_Terms_Approved    9791 non-null   int64  
 3   Bureaucratic_Code_0_Approval        9791 non-null   int64  
 4   Bureaucratic_Code_0_Approved        9791 non-null   int64  
 5   Total_Amount                        9791 non-null   float64
 6   Region                              9791 non-null   int8   
 7   Territory                           9791 non-null   int8   
 8   ASP_(converted)                     9791 non-null   float64
 9   Account_Type                        9791 non-null   int8   
 10  Delivery_Quarter                    9791 non-null   int8   
 11  Product_Type                        9791 n

In [83]:

res = xgboost(agrupacionEntrenamientoXGB,entrenamientoResultado,listaDeCategoricas,agrupacionTesteoXGB)
res

array([[0.06901902, 0.930981  ],
       [0.38701797, 0.61298203],
       [0.0403347 , 0.9596653 ],
       ...,
       [0.92158395, 0.07841606],
       [0.01706135, 0.98293865],
       [0.8929871 , 0.10701291]], dtype=float32)

In [84]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.930981  , 0.61298203, 0.9596653 , ..., 0.07841605, 0.98293865,
       0.10701293], dtype=float32)

In [85]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.93
10690,0.61
10691,0.96
10692,0.44
10693,1.00


## LightGBM

In [ ]:
def lightGBM(train_data,train_labels,cat_features,eval_data):
  # evaluate the model
  model = lgb.LGBMClassifier()
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model,
                             train_data,
                             train_labels,
                             scoring='accuracy',
                             cv=cv,
                             n_jobs=-1,
                             categorical_feature=cat_features,
                             error_score='raise')
  print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
  # fit the model on the whole dataset
  model = lgb.LGBMClassifier()
  model.fit(X, y)
  # make a single prediction
  yhat = model.predict(eval_data)
  return yhat
"""
  parameters = {
      'application': 'binary',
      'objective': 'binary',
      'metric': 'auc',
      'is_unbalance': 'true',
      'boosting': 'gbdt',
      'num_leaves': 31,
      'feature_fraction': 0.5,
      'bagging_fraction': 0.5,
      'bagging_freq': 20,
      'learning_rate': 0.05,
      'verbose': 0,
      'categorical_feature':cat_features
  }

  model = lgb.train(parameters,
                        train_data,
                        valid_sets=train_labels,
                        num_boost_round=5000,
                        early_stopping_rounds=100)

  x = eval_data
  y = model.predict(x)
  output = pd.DataFrame({'id': ids, 'target': y})
"""
  #output.to_csv("submission.csv", index=False)
  # save model to file
  #gbm.save_model('model.txt')

"\n  parameters = {\n      'application': 'binary',\n      'objective': 'binary',\n      'metric': 'auc',\n      'is_unbalance': 'true',\n      'boosting': 'gbdt',\n      'num_leaves': 31,\n      'feature_fraction': 0.5,\n      'bagging_fraction': 0.5,\n      'bagging_freq': 20,\n      'learning_rate': 0.05,\n      'verbose': 0,\n      'categorical_feature':cat_features\n  }\n\n  model = lgb.train(parameters,\n                        train_data,\n                        valid_sets=train_labels,\n                        num_boost_round=5000,\n                        early_stopping_rounds=100)\n\n  x = eval_data\n  y = model.predict(x)\n  output = pd.DataFrame({'id': ids, 'target': y})\n"

In [ ]:
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter','Billing_Country']
res = lightGBM(agrupacionEntrenamiento,entrenamientoResultado,listaDeCategoricas,agrupacionTesteo)
res

TypeError: ignored

## Ramdom forest

In [169]:
def ramdomForest(train_data,train_labels,eval_data):
  # Crear el modelo
  model = RandomForestClassifier(n_estimators=200,
                                 #max_features=5,
                                 min_samples_leaf=1,
                                 oob_score=10,
                                 bootstrap=True,
                                 verbose=3,
                                 max_features='sqrt')
  #parametros importantes
  #  n_estimators: será la cantidad de árboles que generaremos.
  #  max_features: la manera de seleccionar la cantidad máxima de features para cada árbol.
  #  min_sample_leaf: número mínimo de elementos en las hojas para permitir un nuevo split (división) del nodo.
  #  oob_score: es un método que emula el cross-validation en árboles y permite mejorar la precisión y evitar overfitting.
  #  boostrap: para utilizar diversos tamaños de muestras para entrenar. Si se pone en falso, utilizará siempre el dataset completo.
  #  n_jobs: si tienes multiples cores en tu CPU, puedes indicar cuantos puede usar el modelo al entrenar para acelerar el entrenamiento.
  # a entrenar!
  model.fit(train_data, train_labels)
  predictions = model.predict_proba(eval_data)
  return predictions
  

In [189]:
#Uso los mismos valores de XGB para las variables categoricas ya que no las soporta
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
res = ramdomForest(agrupacionEntrenamientoXGB.drop(desacerse,axis=1),
                   entrenamientoResultado,
                   agrupacionTesteoXGB.drop(desacerse,axis=1))
res

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


array([[0.325, 0.675],
       [0.43 , 0.57 ],
       [0.47 , 0.53 ],
       ...,
       [0.565, 0.435],
       [0.445, 0.555],
       [0.73 , 0.27 ]])

In [190]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.675, 0.57 , 0.53 , ..., 0.435, 0.555, 0.27 ])

In [191]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.68
10690,0.57
10691,0.53
10692,0.54
10693,0.68
